<a href="https://colab.research.google.com/github/AnahitShekikyan/ADS-506-Final-Team-Project/blob/main/ADS_506_Final_Project_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bike Sharing Dataset**

---



# **Import Data & Libraries**

In [ ]:
# Import libraries
!pip install pmdarima
import pmdarima as pm
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#importing data
from google.colab import drive
drive.mount('/content/drive')
df_day = pd.read_csv('/content/drive/MyDrive/day.csv')
df_hour = pd.read_csv('/content/drive/MyDrive/hour.csv')

# **Basic Data Information**

In [ ]:
# printing information about data
df_day.info()
df_hour.info()

In [ ]:
# Checking for duplicates
df_day.duplicated().sum()
df_hour.duplicated().sum()

In [ ]:
# printing basic describtion about day and hour datasets
df_day.describe()
df_hour.describe()

In [ ]:
# converting date to numeric
df_day['dteday'] = pd.to_datetime(df_day['dteday'])
df_hour['dteday'] = pd.to_datetime(df_hour['dteday'])

# adding new features like day of the year
df_hour['day_of_year'] = df_hour['dteday'].dt.dayofyear

# **Data Visualization and Distribution Analysis**

In [ ]:
# Distribution of histogram of the "cnt" variable (overall pattern of bike usage)
sns.histplot(df_hour['cnt'], kde=True)
plt.title("Distribution of Bike Rentals")
plt.xlabel("Count of Bike Rentals")
plt.show()

In [ ]:
# Scatterplot of temperature, weather, and bike rentals
sns.scatterplot(data=df_day, x='temp', y='cnt', hue='weathersit')
plt.title("Temperature vs. Bike Rentals")
plt.xlabel("Temperature")
plt.ylabel("Count of Bike Rentals")
plt.show()

# **Correlation Analysis**

In [ ]:
# Creating the correlation matrix
corr = df_hour.drop(columns=['dteday']).corr()

# Creating a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Creating the heatmap with the mask
plt.figure(figsize=(10, 8))
sns.heatmap(corr, mask=mask, annot=True, cmap='coolwarm', annot_kws={"size": 8})
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.title('Triangular Correlation Heatmap')
plt.show()

# **Temporal Analysis**

In [ ]:
# showing how the number of rentals varies throughout the day by aggregating the "cnt" by the "hr" column
sns.lineplot(data=df_hour.groupby('hr')['cnt'].sum().reset_index(), x='hr', y='cnt', marker='o') #Applying sum() before reset_index()
plt.title('Average Bike Rentals by Hour')
plt.xlabel('Hour of Day')
plt.ylabel('Average Count of Rentals')
plt.show()

In [ ]:
# Evaluating monthly trends for the seasonal effects
monthly_trend = df_hour.groupby('mnth')['cnt'].mean().reset_index()
sns.lineplot(x='mnth', y='cnt', data=monthly_trend, marker='o')
plt.title('Average Bike Rentals by Month')
plt.xlabel('Month')
plt.ylabel('Average Count of Rentals')
plt.show()

# **Categorical Feature Analysis**

In [ ]:
# bike rentals across different seasons ("season") or weather situations ("weathersit")
sns.boxplot(x='season', y='cnt', data=df_day, palette='viridis')
plt.title('Bike Rentals by Season')
plt.xlabel('Season (1=Spring, 2=Summer, 3=Fall, 4=Winter)')
plt.ylabel('Count of Rentals')
plt.show()

In [ ]:
# Comparing the average rentals during holidays (holiday == 1) vs non-holidays
sns.boxplot(x='holiday', y='cnt', data=df_day)
plt.title('Bike Rentals: Holiday vs Non-Holiday')
plt.xlabel('Holiday')
plt.ylabel('Count of Rentals')
plt.show()


# **Seasonality Decomposition**

In [ ]:
# Useing seasonal decomposition to break down the time series into trend, seasonal, and residual components

result = seasonal_decompose(df_day['cnt'], model='additive', period=12)
result.plot()
plt.show()

# **Outlier Detection**

In [ ]:
# detect any anomalies in the dataset
sns.boxplot(data=df_hour[['temp', 'hum', 'windspeed', 'cnt']])
plt.title('Outlier Detection in Features')
plt.show()

In [ ]:
# Removing extreme outliers (values beyond 3 times the IQR)
Q1 = df_day['cnt'].quantile(0.25)
Q3 = df_day['cnt'].quantile(0.75)
IQR = Q3 - Q1
df_day_no_outliers = df_day[(df_day['cnt'] >= (Q1 - 3 * IQR)) & (df_day['cnt'] <= (Q3 + 3 * IQR))]

In [ ]:
# Exploring Feature Relationships
sns.pairplot(df_hour[['temp', 'atemp', 'hum', 'windspeed', 'cnt']])
plt.show()

# **Feature Engineering Ideas**

In [ ]:
# Convert date columns to datetime
df_day['dteday'] = pd.to_datetime(df_day['dteday'])
df_hour['dteday'] = pd.to_datetime(df_hour['dteday'])

In [ ]:
# Train-Test Split
train_size = int(len(df_day) * 0.8)  # 80% training data
train, test = df_day.iloc[:train_size], df_day.iloc[train_size:]

In [ ]:
# Setting date index for time series compatibility
train.set_index('dteday', inplace=True)
test.set_index('dteday', inplace=True)

# **Metrics and Evalulations**

In [ ]:
# Stationarity Test (ADF Test)
adf_test = adfuller(df_day['cnt'])
print(f"ADF Statistic: {adf_test[0]}")
print(f"p-value: {adf_test[1]}")
print(f"Critical Values: {adf_test[4]}")

In [ ]:
# Differencing if Non-Stationary
df_day['cnt_diff'] = df_day['cnt'].diff().dropna()
plt.plot(df_day['cnt_diff'])
plt.title("Differenced Bike Rentals Time Series")
plt.show()

In [ ]:
# Random Walk Baseline
test['random_walk'] = train['cnt'].iloc[-1]
rmse_rw = np.sqrt(mean_squared_error(test['cnt'], test['random_walk']))
mae_rw = mean_absolute_error(test['cnt'], test['random_walk'])

print(f"Random Walk RMSE: {rmse_rw:.2f}, MAE: {mae_rw:.2f}")

In [ ]:
test['forecast'] = train['cnt'].iloc[-1]

# Assuming 'test' DataFrame contains 'cnt' (actual) and 'forecast' (predicted) values
residuals = test['cnt'] - test['forecast']

# Plot ACF for the residuals
plt.figure(figsize=(8, 6))
plot_acf(residuals.dropna(), lags=40, title="ACF of Residuals")
plt.show()

# Plot PACF for the residuals
plt.figure(figsize=(8, 6))
plot_pacf(residuals.dropna(), lags=40, title="PACF of Residuals")
plt.show()

In [ ]:
# Train the ARIMA model
arima_model = ARIMA(train['cnt'], order=(1, 1, 1))
arima_result = arima_model.fit()

# Residual analysis for manually tuned ARIMA
arima_residuals = arima_result.resid

# Plot residuals
plt.figure(figsize=(8, 6))
plt.plot(arima_residuals, label="ARIMA Residuals", color= "blue")
plt.axhline(0, linestyle="--", color="red", linewidth=1)
plt.title("ARIMA Residual Analysis")
plt.xlabel("Time")
plt.ylabel("Residuals")
plt.legend()
plt.show()

# ACF of ARIMA residuals
plt.figure(figsize=(8, 6))
plot_acf(arima_residuals.dropna(), lags=40)
plt.title("ACF of ARIMA Residuals")
plt.xlabel("Lags")
plt.ylabel("Autocorrelation")
plt.show()

# Histogram of Residuals
plt.figure(figsize=(8, 6))
sns.histplot(arima_residuals, kde=True, color="purple")
plt.title("Histogram of Residuals - ARIMA Model")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.axvline(0, linestyle="--", color="red", linewidth=1)
plt.show()

# Normal Q-Q Plot for Residuals
sm.qqplot(arima_residuals, line="s", color="green")
plt.title("Q-Q Plot of Residuals - ARIMA Model")
plt.show()

# **Model 1: ETS (Exponential Smoothing)**

In [ ]:
# Fit ETS Model
ets_model = ExponentialSmoothing(train['cnt'], seasonal='add', trend='add', seasonal_periods=12).fit()

# Forecast
ets_forecast = ets_model.forecast(len(test))

# Evaluate ETS model
ets_mse = mean_squared_error(test['cnt'], ets_forecast)
print(f"ETS - MSE: {ets_mse:.2f}, MAE: {ets_mae:.2f}")

# **Model 2: TSLM (Time Series Linear Model)**

In [ ]:
# Time as a feature
train['time'] = range(len(train))
test['time'] = range(len(train), len(train) + len(test))

# Create Polynomial Features
poly = PolynomialFeatures(degree=2, include_bias=False)
train_features = poly.fit_transform(train[['time']])
test_features = poly.transform(test[['time']])

# Fit Linear Model
tslm_model = LinearRegression()
tslm_model.fit(train_features, train['cnt'])

# Forecast
tslm_forecast = tslm_model.predict(test_features)

# Evaluate TSLM
tslm_mse = mean_squared_error(test['cnt'], tslm_forecast)
tslm_mae = mean_absolute_error(test['cnt'], tslm_forecast)
print(f"TSLM - MSE: {tslm_mse:.2f}, MAE: {tslm_mae:.2f}")

# **Model 3: NNETAR (Neural Network Autoregression)**

In [ ]:
# Set random seeds
np.random.seed(123)
tf.random.set_seed(123)

# Prepare data for Neural Network
lag = 12  # Number of lags
X_train = np.array([train['cnt'].iloc[i - lag:i].values for i in range(lag, len(train))])
y_train = train['cnt'].iloc[lag:].values

# Build NNETAR model
nnetar_model = Sequential()
nnetar_model.add(Dense(32, activation='relu', input_shape=(lag,)))
nnetar_model.add(Dense(16, activation='relu'))
nnetar_model.add(Dense(1))
nnetar_model.compile(optimizer='adam', loss='mse')

# Train NNETAR model
nnetar_model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)

# Forecast with NNETAR
X_test = np.array([test['cnt'].iloc[i - lag:i].values for i in range(lag, len(test))])
nnetar_forecast = nnetar_model.predict(X_test)

# Evaluate NNETAR
nnetar_mse = mean_squared_error(test['cnt'][lag:], nnetar_forecast)
nnetar_mae = mean_absolute_error(test['cnt'][lag:], nnetar_forecast)
print(f"NNETAR - MSE: {nnetar_mse:.2f}, MAE: {nnetar_mae:.2f}")

# **Model 4: ARIMA**

In [ ]:
# Train ARIMA model with manually specified parameters
arima_model = ARIMA(train['cnt'], order=(1, 1, 1))
arima_result = arima_model.fit()

# Forecast using ARIMA
arima_forecast = arima_result.forecast(steps=len(test))

# Evaluate ARIMA
arima_mse = mean_squared_error(test['cnt'], arima_forecast)
arima_mae = mean_absolute_error(test['cnt'], arima_forecast)
print(f"ARIMA Model - MSE: {arima_mse:.2f}, MAE: {arima_mae:.2f}")

In [ ]:
# Plot ARIMA forecast vs. actual
plt.figure(figsize=(8, 6))
plt.plot(test.index, test['cnt'], label="Actual")
plt.plot(test.index, arima_forecast, label="ARIMA Forecast", linestyle="--")
plt.legend()
plt.title("ARIMA Forecast vs Actual")
plt.show()

# **Model 5: Auto-ARIMA (SARIMA with Automated Parameter Selection)**

In [ ]:
# Find optimal parameters using Auto-ARIMA
auto_model = pm.auto_arima(train['cnt'], seasonal=True, m=12, suppress_warnings=True, stepwise=True)

print(auto_model.summary())

# Extract the optimal parameters
auto_order = auto_model.order
auto_seasonal_order = auto_model.seasonal_order

# Train SARIMA with Auto-ARIMA parameters
sarima_model = SARIMAX(train['cnt'], order=auto_order, seasonal_order=auto_seasonal_order)
sarima_result = sarima_model.fit()

# Forecast using Auto-ARIMA
sarima_forecast = sarima_result.predict(start=test.index[0], end=test.index[-1])

# Evaluate Auto-ARIMA
sarima_mse = mean_squared_error(test['cnt'], sarima_forecast)
sarima_mae = mean_absolute_error(test['cnt'], sarima_forecast)

print(f"Auto-ARIMA Model - MSE: {sarima_mse:.2f}, MAE: {sarima_mae:.2f}")

In [ ]:
# Plot Auto-ARIMA forecast vs. actual
plt.figure(figsize=(8, 6))
plt.plot(test.index, test['cnt'], label="Actual")
plt.plot(test.index, sarima_forecast, label="Auto-ARIMA Forecast", linestyle="--")
plt.legend()
plt.title("Auto-ARIMA (SARIMA) Forecast vs Actual")
plt.show()

# **Visualization of Results**

In [ ]:
# Plot all forecasts
plt.figure(figsize=(8, 6))
plt.plot(train['cnt'], label='Train')
plt.plot(test['cnt'], label='Test')
plt.plot(test.index, ets_forecast, label='ETS Forecast', linestyle='--')
plt.plot(test.index, tslm_forecast, label='TSLM Forecast', linestyle=':')
plt.plot(test.index[lag:], nnetar_forecast, label='NNETAR Forecast', linestyle='-.')
plt.plot(test.index, arima_forecast, label='ARIMA Forecast', linestyle='-')
plt.plot(test.index, sarima_forecast, label='Auto-ARIMA Forecast', linestyle='--')
plt.legend()
plt.title('Model Forecasts')
plt.show()


# **Summary Table**

In [ ]:
# Metrics for all models (use the values from your analysis)
summary_data = {
    "Model": ["ETS", "TSLM", "NNETAR", "ARIMA", "Auto-ARIMA"],
    "Mean Squared Error (MSE)": [ets_mse, tslm_mse, nnetar_mse, arima_mse, sarima_mse],
    "Mean Absolute Error (MAE)": [ets_mae, tslm_mae, nnetar_mae, arima_mae, sarima_mae],
    "Remarks": [
        "Captured seasonality well.",
        "Overfitted to data trends.",
        "Best performance with non-linearity.",
        "Performed adequately for time series.",
        "Streamlined parameter tuning but similar to ARIMA."
    ]
}

# Convert the data to a DataFrame
summary_df = pd.DataFrame(summary_data)

display(summary_df)